In [19]:
import numpy as np

In [1]:
#code for making artificial dataset
import random
def random_string(length,alphabet_list):
    rand_str = ''.join(random.choice(alphabet_list) for i in range(length))
    return rand_str

def perturb(seed,alphabet_list,p=0.5):
    seq=''
    for c in seed:
        if random.random() < p: c = random.choice(alphabet_list)
        seq += c
    return seq

def make_artificial_dataset(alphabet='ACGT', motives=None, motif_length=6, 
                            sequence_length=100, n_sequences=1000, n_motives=2, p=0.2,
                           random_state=1):
    random.seed(random_state)
    alphabet_list=[c for c in alphabet]
    
    if motives is None:
        motives=[]
        for i in range(n_motives):
            motives.append(random_string(motif_length,alphabet_list))
    else:
        motif_length = len(motives[0])
        n_motives = len(motives)
    
    sequence_length = sequence_length / len(motives)
    flanking_length = (sequence_length - motif_length ) / 2
    n_seq_per_motif = n_sequences

    counter=0
    seqs=[]
    binary_seqs=[]
    for i in range(n_seq_per_motif):
        total_seq = ''
        total_binary_seq=''
        for j in range(n_motives):
            left_flanking = random_string(flanking_length,alphabet_list)
            right_flanking = random_string(flanking_length,alphabet_list)
            noisy_motif = perturb(motives[j],alphabet_list,p)
            seq = left_flanking + noisy_motif + right_flanking
            total_seq += seq
            binary_seq = '0' * flanking_length + '1' * motif_length + '0' * flanking_length
            total_binary_seq += binary_seq
        seqs.append(('>ID%d'%counter,total_seq))
        binary_seqs.append(('>ID%d'%counter,total_binary_seq))
        counter += 1
    return motives, seqs, binary_seqs

In [2]:
#setup parameters
alphabet='ACGT'
semi_len=9
motives=['A'*semi_len+'C'*semi_len,
         'C'*semi_len+'A'*semi_len,
         'A'*semi_len+'T'*semi_len,
         'T'*semi_len+'A'*semi_len]
sequence_length=60
n_sequences=50
p=0.2

#make dataset
motives, seqs, binary_seqs = make_artificial_dataset(alphabet=alphabet,
                                        motives=motives,
                                        sequence_length=sequence_length,
                                        n_sequences=n_sequences,
                                        p=p,
                                        random_state=8)

In [ ]:
def create_data(motif_length=10,
                n_motif=4,
                n_sequences=50,
                sequence_length=30,)

In [4]:
from sklearn.cluster import KMeans
from utilities import Weblogo, MuscleAlignWrapper
from eden_wrapper import EdenWrapper
from meme_wrapper import Meme

In [5]:
# size of context
# size of motif
# number of motifs
# number of sequences
# 

In [8]:
motif_length = 6
sequence_length = 50
n_sequences = 50
p=0.2

for i in range(6,8):
    n_motives=i
    motives, seqs, binary_seqs = make_artificial_dataset(alphabet='ACGT',
                                                         motif_length=motif_length,
                                                         n_motives=n_motives,
                                                         sequence_length=sequence_length,
                                                         n_sequences=n_sequences,
                                                         p=p,
                                                         random_state=8)
    km = KMeans(n_clusters=n_motives)
    tool = EdenWrapper(alphabet='dna', 
                       complexity=5, 
                       nbits=14, 
                       negative_ratio=3,
                       min_subarray_size=len(motives[0])-2, 
                       max_subarray_size=len(motives[0])+2,
                       clustering_algorithm=km
                      )
    tool.fit(seqs)

AttributeError: 'EdenWrapper' object has no attribute 'pwms_list'

In [10]:
scores = [[] for i in range(n_motives)]
print scores

[[], [], [], [], [], []]


In [12]:
a = np.array([1, 2, 3])
b = np.array([2, 3, 4])
c=np.vstack((a,b))


In [22]:
scores[1]=[[1]]
print scores

[[], [[1]], [], [], [], []]


In [14]:
np.mean(c, axis=0)

array([ 1.5,  2.5,  3.5])

In [17]:
c=np.vstack((c,[7,8,9]))

In [18]:
c

array([[1, 2, 3],
       [2, 3, 4],
       [1, 2, 3],
       [2, 3, 4],
       [7, 8, 9]])